# Functions

## Basics

The built-in Julia functions like `print` you can use without even being aware that they are functions. Defining new ones can be done with the `function` keyword, starting a block that ends with `end`.

In [1]:
function sayHi()
    println( "Hi." )
end

sayHi()

Hi.


Arguments are passed much like they are in other languages. No type has to be specified (dynamic typing).

In [2]:
function sayHi( name )
    println( "Hi, $(name)." )
end

sayHi( "dude" )

Hi, dude.


Note that the first definition is not overwritten. Rather, as it has a different signature (no arguments, to be precise). You can still call it.

In [3]:
sayHi()

Hi.


Forcing types is possible for *local* variables such as functional arguments and function bodies. Append `::` followed by the desired type to assert it's the one you were expecting.

In [4]:
function sayHi( name::String )
    println( "Hi, $(name)." ) 
end

sayHi( "dude" )

Hi, dude.


This will not work for *global* variables (at least for now). It makes little sense anyway.

In [5]:
a::Float32 = 10.0

LoadError: syntax: type declarations on global variables are not yet supported

Because all three versions of `sayHi` can be called separately, you can reuse their functionality. Notice the message the kernel produces when defining a new `sayHi`.

In [6]:
function sayHi( name::String, num::Int64 )
    for i in 1:num
        sayHi( name )
    end
end

sayHi (generic function with 4 methods)

Thanks to magic of multiple dispatch, `sayHi` is treated as a *generic* function that has four methods with different signatures.

In [7]:
sayHi( "dude", 4 )

Hi, dude.
Hi, dude.
Hi, dude.
Hi, dude.


You can get a listing of all methods and where they were declared.

In [8]:
methods(sayHi)

# 4 methods for generic function "sayHi":
[1] sayHi() in Main at In[1]:1
[2] sayHi(name::String) in Main at In[4]:1
[3] sayHi(name::String, num::Int64) in Main at In[6]:1
[4] sayHi(name) in Main at In[2]:1

Lastly, you can specify default arguments where necessary.

In [9]:
function sayHi( name="Dude" )
    println( "Hi, $(name)." )
end

sayHi( )

Hi, Dude.


## Returning values

The keyword `return` passes back the last computed value for the expression and turns control over to the caller. Julia evaluates nothing after `return`.

In [10]:
function add(apples, oranges)
    sum = apples + oranges
    return sum
    
    sum = 0
end

add( 1, 2 ) # returns 3 instead of 0

3

If you would like to make sure your function returns a specific type, you may specify this explicitly. They are not global variables, so you can use `::` to assert type. Note how in the following declaration both the arguments to the function and its return type are now fixed.

In [11]:
function add(apples::Int8, oranges::Int8)::Int8
    return apples + oranges
end

add( 1, 2)

3

`return` may not even be necessary unless you need it to control flow. Functions return their last evaluated expression by default.

In [12]:
function add(apples, oranges)
    apples + oranges
end

add( 1, 2 ) # 3

3

As another example, here's a textbook classic: a recursive function to calculate factorials.

In [13]:
## explicit
function factorial( n::Int64 )::Int64
    result = n
    if n>1
        result = result * factorial(n-1)
    else
        return result
    end
end

factorial(39)

2304077777655037952

Briefer, without the `else` clause:

In [14]:
function factorial( n::Int64 )::Int64
    result = n
    if n>1
        result = result * factorial(n-1)
    end
    
    result
end

factorial(39)

2304077777655037952

Calling `return` without anything else technically returns `nothing`. It is the implied form of `return nothing`.

In [15]:
function add(apples, oranges)
    apples + oranges
    return
end

add( 1, 2 ) # <cricket noise>

Operators such as `+` and `-` are functions at heart and you can overload them to handle custom data types. If you wanted to keep track of your fruit basket, you might define `struct`s for each type.

In [16]:
struct Apple
    name::String
    count::Int
end

myApples = Apple( "Golden Delicious", 6 )
moreApples = Apple( "Granny Smith", 6 );

To find how many apples you have, you could not simply type:

In [17]:
myApples + moreApples

LoadError: MethodError: no method matching +(::Apple, ::Apple)
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:560

Overload the base operator `+` to make turn this into a valid statement.

In [18]:
import Base.+

function +(a::Apple, b::Apple)
    a.count + b.count
end

@show myApples + moreApples

myApples + moreApples = 12


12

Now here is a type for oranges. 

In [19]:
struct Orange
    name::String
    count::Int
end

myOranges = Orange( "Valencia", 4 )

Orange("Valencia", 4)

Needless to say, even though they have the same fields, attempting to add them will throw an error. There is simply no function to add `Apple` and `Orange`.

In [20]:
myApples + myOranges

LoadError: MethodError: no method matching +(::Apple, ::Orange)
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:560
[0m  +(::Apple, [91m::Apple[39m) at In[18]:3

**Aside**

Here is an elegant way to accomplish this anyway. It uses Julias hierarchical typing. You need only create a supertype `Fruit` (using `<:`) and redefine `Apple` and `Orange`.

```
abstract type Fruit end

struct Apple <: Fruit
    name::String
    count::Int
end

struct Orange <: Fruit
    name::String
    count::Int
end
```

Do another overload of `+` for type `Fruit`.

```
import Base.+

function +(a::Fruit, b::Fruit)
    a.count + b.count
end
```

And now Julia will happily add `Apple` and `Orange`.
```
myApples = Apple( "Golden Delicious", 6 )
moreApples = Apple( "Granny Smith", 6 )
myOranges = Orange( "Valencia", 4 );

@show myApples + myOranges
# myApples + myOranges = 10
```

## Anonymous functions

Another core element of Julia, sometimes referred to as *lambda functions* in languages such as Python. Unlike in other programming environments, they are more deeply integrated into Julia. You can pass them as arguments, and return them. Frequently they are mapped over expressions for transformations. `map` is what is known as a *higher-order function* in that it not only operates on arguments but also on functions.

Mapping a function works like so:

In [21]:
map( sqrt, [1, 2, 3, 4] ) # invoke sqrt() with the four elements given in the array

4-element Vector{Float64}:
 1.0
 1.4142135623730951
 1.7320508075688772
 2.0

Calculating the volume of a sphere is not built-in. Here's a function for this. Notice the following:
* while not necessary because of dynamic typing, both the argument and the return type of `sphereVolume` are typed
* function asserts `r`is a number, so it can an int or a float
* it says $\pi$, not `pi` 

In [22]:
function sphereVolume( r::Number )::Float64
    return 4/3*π*r^3
end

sphereVolume( 10 )

4188.790204786391

In [23]:
map( sphereVolume, [1, 2, 3, 4] ) # Julia knows from the signature that it should pass the elements of the array to r

4-element Vector{Float64}:
   4.1887902047863905
  33.510321638291124
 113.09733552923254
 268.082573106329

It's a simple equation. With an anonymous function, you can cut corners.

In [24]:
map( r -> 4/3*π*r^3, [1,2,3,4] ) # map r to function. No signature, so you need to say where the elements go.

4-element Vector{Float64}:
   4.1887902047863905
  33.510321638291124
 113.09733552923254
 268.082573106329

There is also a block version.

In [25]:
map( [1, 2, 3, 4] ) do r
    sphereVolume(r)
end

4-element Vector{Float64}:
   4.1887902047863905
  33.510321638291124
 113.09733552923254
 268.082573106329

## Closures

A closure is the equivalent of an object with a single method. It's called *closure* because it *encloses*, or *closes over*, a lexical scope. This allows creating functions that can remember their state by taking with them their own scope. Their return value is another function.

Julia uses lexical scoping, meaning that invoking a function does not grant it access to its caller's scope. Accessing a global variable will throw an error. It is a way of avoiding so-called `side effects`, modifications of states / variables, outside of where you may have intended to. For instance, implementing a counter in Python might look like this:

```
counter = 0

def increaseCounter( val ):
    global counter
    counter = counter + val

print( "Counter: ", counter )
increaseCounter(10)
print( "Counter: ", counter )
```

Very similarly in Julia, which also has a `global` keyword.

In [26]:
counter = 0

function increaseCounter( val::Int )
    global counter = counter + val
end

println( "Counter: $(counter)" )
increaseCounter(10) # add 10 
println( "Counter: $(counter)" )

Counter: 0
Counter: 10


Using a closure, the variable `counter` is wrapped in `increaseCounter`.

In [27]:
function increaseCounter()
    counter = 0
    
    function add( val=1 )
        counter = counter + val
    end
end

counter = increaseCounter() # counter == 0
@show counter() # increase by one, counter == 1
@show counter(2); # increase by two, counter == 3

counter() = 1
counter(2) = 3


Counters are a typical use case for closures in JavaScript. As web pages will not be your primary reason for using Julia, consider this function that retrieves an element from an absurdly large array by index `idx`. Call this repeatedly to see

* consistently long run times
* memory overhead, including time wasted with garbage collections (*gc time*)

In [28]:
function getElement( idx )
    absurdArray = collect(1:1e8)
    
    return absurdArray[idx]
end

@time getElement( 10 )
@time getElement( 100 )
@time getElement( 1000 )
@time getElement( 10000 );

  0.390265 seconds (2 allocations: 762.940 MiB, 1.07% gc time)
  0.412359 seconds (2 allocations: 762.940 MiB, 6.93% gc time)
  0.424776 seconds (2 allocations: 762.940 MiB, 8.74% gc time)
  0.448940 seconds (2 allocations: 762.940 MiB, 14.20% gc time)


You can the re-allocations and garbage collecting by using a closure because now the local variable (the absurdly large array) is stored with the function.

In [29]:
function getElementClosure(  )
    absurdArray = collect(1:1e8)
    
    function getElement( idx )
         absurdArray[idx]
    end
end

@time closure = getElementClosure()
@time closure( 10 )
@time closure( 100 )
@time closure( 1000 )
@time closure( 10000 );

  0.440292 seconds (3 allocations: 762.940 MiB, 12.59% gc time)
  0.002316 seconds (826 allocations: 56.795 KiB, 97.77% compilation time)
  0.000002 seconds (1 allocation: 16 bytes)
  0.000001 seconds (1 allocation: 16 bytes)
  0.000000 seconds (1 allocation: 16 bytes)


Run times drop drastically after the initial assignment of `getElementClosure` to the variable `closure`.